## Introduccion

Hoy veremos un poco mas como leer fichieros netcdfs (Network Common Data Format). Este formato ha sido devenvolupado por la communidad UNIDATA. Nos enfocaremos en este formato de fichiero y no trataremos otros typos de fichieros como ascii o csv, porque son mas facil de leer (con una simple busqueda a google encontrareis lo que necessitais) y porque son menos y menos utilizados en meteorologia, oceanografia y clima.


El netcdf presenta muchas ventajas comparado a otros formatos de fichieros:

    - Los fichieros NetCDF son fichieros binarios, es decir que no se poden leer sin librerias o software adecuados, pero ocupan menos espacio en la memoria que los mismos datos en csv o ascii.
    - formato portable: los datos puede seer leido en todos ordenadores, incluso que su manera de guardar enteros, floats o caracteres sea distinta.
    - fichieros autodescritos que permiten a los softwares de NetCDFs conocer directamente la estructura, el nombre de las variables, la unidades, la dimensiones.... eso significa que el usuario no necessita un conocimiento previo y eso reduce el riesgo de error
    - Existen librerias y programas numerosos para leer estos fichieros.
    - se puede facilmente modificar el fichiero añadiendo o quitando datos.
    - las librerias son gratuitas.
    - There are convenient and easy to use.

In [3]:
#import netcdf library
from netCDF4 import Dataset as nc 
#import date conversor netcdf
from netCDF4 import num2date, date2num 
from datetime import date, timedelta, datetime
import numpy as np
import os

## Descarregar fichiero

Descargaremos unos datos de la renalysis NCEP: https://www.esrl.noaa.gov/psd/data/gridded/data.ncep.reanalysis.html
las reanalysis son datos de modelos en los cual estan assimilados obervaciones, de esta manera permite reconstruir variables dificiles de medir, como el viento, la altura de geopotential, la humedad de suelo, corriente marinos, hielos marino... pero se tienen que utilizar con cautela, no son obervaciones y tambien pueden tener cesgos importantes.

Descarecaremos los monthly values del geopotential por eso, ir en la pagina: https://www.esrl.noaa.gov/psd/data/gridded/data.ncep.reanalysis.html  buscar pressure level, y el geopotential con frequencia mensual. Se os descarregara un fichiero nombrado: `hgt.mon.mean.nc`.

Para simplificar nos la vida guarderemos este fichiero en la carpeta del notebook. Pero eso no es la mejor manera lo veremos luego.

## Leer fichiero - cabesera

Primero abrimos el fichiero.

In [7]:
filename = "https://www.esrl.noaa.gov/psd/thredds/dodsC/Datasets/ncep.reanalysis.derived/pressure/hgt.mon.mean.nc"
filegeo = nc(filename, "r") # open the netcdf file

una vez el fichiero abierto lo primero que tenemos que hacer siempre es mirar la informacion que contiene.

In [8]:
print(filegeo)

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF3_CLASSIC data model, file format DAP2):
    description:  Data from NCEP initialized reanalysis (4x/day).  These are interpolated to pressure surfaces from model (sigma) surfaces.
    platform: Model
    Conventions: COARDS
    NCO: 20121012
    history: Created by NOAA-CIRES Climate Diagnostics Center (SAC) from the NCEP
reanalysis data set on 07/07/97 by calc.mon.mean.year.f using
/Datasets/nmc.reanalysis.derived/pressure/hgt.mon.mean.nc
from /Datasets/nmc.reanalysis/pressure/hgt.79.nc to hgt.95.nc
Converted to chunked, deflated non-packed NetCDF4 2014/09
    title: monthly mean hgt from the NCEP Reanalysis
    References: http://www.esrl.noaa.gov/psd/data/gridded/data.ncep.reanalysis.derived.html
    dataset_title: NCEP-NCAR Reanalysis 1
    DODS_EXTRA.Unlimited_Dimension: time
    dimensions(sizes): time(861), lat(73), level(17), lon(144)
    variables(dimensions): float32 level(level), float32 lon(lon), float64 time(time), floa

vemos que contiene unos datos de geopotential que contiene 5 variables: level, lat, lon, time and hgt (el geopotential).

In [11]:
print(filegeo.description)

 Data from NCEP initialized reanalysis (4x/day).  These are interpolated to pressure surfaces from model (sigma) surfaces.


In [12]:
print(filegeo.dimensions)

OrderedDict([('time', <class 'netCDF4._netCDF4.Dimension'> (unlimited): name = 'time', size = 861
), ('lat', <class 'netCDF4._netCDF4.Dimension'>: name = 'lat', size = 73
), ('level', <class 'netCDF4._netCDF4.Dimension'>: name = 'level', size = 17
), ('lon', <class 'netCDF4._netCDF4.Dimension'>: name = 'lon', size = 144
)])


In [13]:
print(filegeo.variables)

OrderedDict([('level', <class 'netCDF4._netCDF4.Variable'>
float32 level(level)
    units: millibar
    long_name: Level
    positive: down
    GRIB_id: 100
    GRIB_name: hPa
    actual_range: [1000.   10.]
    axis: Z
unlimited dimensions: 
current shape = (17,)
filling off
), ('lon', <class 'netCDF4._netCDF4.Variable'>
float32 lon(lon)
    units: degrees_east
    long_name: Longitude
    actual_range: [  0.  357.5]
    standard_name: longitude
    axis: X
unlimited dimensions: 
current shape = (144,)
filling off
), ('time', <class 'netCDF4._netCDF4.Variable'>
float64 time(time)
    long_name: Time
    delta_t: 0000-01-00 00:00:00
    avg_period: 0000-01-00 00:00:00
    prev_avg_period: 0000-00-01 00:00:00
    standard_name: time
    axis: T
    units: hours since 1800-01-01 00:00:0.0
    actual_range: [1297320. 1925544.]
    _ChunkSizes: 1
unlimited dimensions: time
current shape = (861,)
filling off
), ('lat', <class 'netCDF4._netCDF4.Variable'>
float32 lat(lat)
    units: degrees_

Las dimensiones et la variables estan guardadas en dos dictionarios, y podemos leer mas informacion sobre estas variables (unit, range, nombre....)

para conocer la dimensiones:

In [14]:
# ver la lista de dimensiones
print(fileTmax.dimensions.keys())
#obtener information de una dimension
print(fileTmax.dimensions.get("level"))
#obtener su tamaño
print(fileTmax.dimensions.get("level").size)



odict_keys(['time', 'lat', 'level', 'lon'])
<class 'netCDF4._netCDF4.Dimension'>: name = 'level', size = 17

17


Y para las variables

In [16]:
print(filegeo.variables.keys())
#obtener information de una variable
print(filegeo.variables.get("hgt"))
#conocer las dimensiones de una variable

odict_keys(['level', 'lon', 'time', 'lat', 'hgt'])
<class 'netCDF4._netCDF4.Variable'>
float32 hgt(time, level, lat, lon)
    long_name: Monthly mean geopotential height
    valid_range: [ -700. 35000.]
    units: m
    add_offset: 0.0
    scale_factor: 1.0
    missing_value: -9.96921e+36
    precision: 0
    least_significant_digit: 0
    GRIB_id: 7
    GRIB_name: HGT
    var_desc: Geopotential height
    level_desc: Multiple levels
    statistic: Mean
    parent_stat: Other
    dataset: NCEP Reanalysis Derived Products
    actual_range: [ -354.45834 32321.098  ]
    _ChunkSizes: [  1   1  73 144]
unlimited dimensions: time
current shape = (861, 17, 73, 144)
filling off



In [17]:
print(fileTmax.variables.get("hgt").shape)

(861, 17, 73, 144)


Mirando el fichiero hemos visto que el geopotential esta guardado en un variable que se llama `hgt` que tiene 4 dimensiones: time, level, lat, lon y un tamaño de 861 (tiempo)x17(level)x73(lat)x144(lon).

## Leer datos

### Tiempo

Ahora nos gustaria leer los datos que hay dentro de este fichiero. Empezaremos por leer el tiempo

In [18]:
# leer variable temporal 
timevar = filegeo.variables["time"]
timevar.units

'hours since 1800-01-01 00:00:0.0'

In [19]:
# para lee los datos hay que poner [:] al final
timevar = filegeo.variables["time"]
timevar[:]


masked_array(data=[1297320., 1298064., 1298760., 1299504., 1300224.,
                   1300968., 1301688., 1302432., 1303176., 1303896.,
                   1304640., 1305360., 1306104., 1306848., 1307520.,
                   1308264., 1308984., 1309728., 1310448., 1311192.,
                   1311936., 1312656., 1313400., 1314120., 1314864.,
                   1315608., 1316280., 1317024., 1317744., 1318488.,
                   1319208., 1319952., 1320696., 1321416., 1322160.,
                   1322880., 1323624., 1324368., 1325040., 1325784.,
                   1326504., 1327248., 1327968., 1328712., 1329456.,
                   1330176., 1330920., 1331640., 1332384., 1333128.,
                   1333824., 1334568., 1335288., 1336032., 1336752.,
                   1337496., 1338240., 1338960., 1339704., 1340424.,
                   1341168., 1341912., 1342584., 1343328., 1344048.,
                   1344792., 1345512., 1346256., 1347000., 1347720.,
                   1348464., 13491

esto esta en hora desde el 1800, no es una unidad muy practica para trabajar... Lo convertiremos en fecha

In [20]:
#y convertir las en fechas
dates = num2date(timevar[:], units=timevar.units) 

In [21]:
dates

array([datetime.datetime(1948, 1, 1, 0, 0),
       datetime.datetime(1948, 2, 1, 0, 0),
       datetime.datetime(1948, 3, 1, 0, 0),
       datetime.datetime(1948, 4, 1, 0, 0),
       datetime.datetime(1948, 5, 1, 0, 0),
       datetime.datetime(1948, 6, 1, 0, 0),
       datetime.datetime(1948, 7, 1, 0, 0),
       datetime.datetime(1948, 8, 1, 0, 0),
       datetime.datetime(1948, 9, 1, 0, 0),
       datetime.datetime(1948, 10, 1, 0, 0),
       datetime.datetime(1948, 11, 1, 0, 0),
       datetime.datetime(1948, 12, 1, 0, 0),
       datetime.datetime(1949, 1, 1, 0, 0),
       datetime.datetime(1949, 2, 1, 0, 0),
       datetime.datetime(1949, 3, 1, 0, 0),
       datetime.datetime(1949, 4, 1, 0, 0),
       datetime.datetime(1949, 5, 1, 0, 0),
       datetime.datetime(1949, 6, 1, 0, 0),
       datetime.datetime(1949, 7, 1, 0, 0),
       datetime.datetime(1949, 8, 1, 0, 0),
       datetime.datetime(1949, 9, 1, 0, 0),
       datetime.datetime(1949, 10, 1, 0, 0),
       datetime.datetime(194

ahora mejor...

### longitude, latitude, level

In [22]:
#leer longitude
lon = filegeo.variables["lon"][:]
#leer latitude
lat = filegeo.variables["lat"][:]
# leer level
lev = filegeo.variables["level"][:]

### geopotential

In [23]:
filegeo.variables["hgt"].shape

(861, 17, 73, 144)

podemos leer todo el `array`:

In [25]:
geo = filegeo.variables["hgt"][:]

RuntimeError: NetCDF: Access failure

In [28]:
from sys import getsizeof
print(getsizeof(geo)) # in bytes
print(getsizeof(geo)*1e-6) #in megabytes

36203480
36.20348


Por ejemplo, nos gustaria trabajar sobre la North Atlantic Oscilation (NAO: https://www.youtube.com/watch?v=KOYJG7j4Iy8). Como lo explica la video, solo nos hace falta dos puntos y la altura de 500hPa.

In [26]:
#Primero buscamos el level correspondiente a 500hPa
print(lev)
# Coo os lo he explicado en clase en mejor nunca utilizar el operador "==" con float
# por temas de precision decimal.
# en ves de escribir 
indlev500 = np.where(lev == 500)[0]
print(indlev500) 
# que en este caso funcciona es mejor buscar el level mas cercano a 500
indlev500 = (np.abs(lev-500)).argmin()
print(indlev500)

[1000.  925.  850.  700.  600.  500.  400.  300.  250.  200.  150.  100.
   70.   50.   30.   20.   10.]
[5]
5


Podemos leer unicamente el level que nos interessa, y va bastante mas rapido, y ocupa menos espacio.

In [ ]:
geo = fielgeo.variables["hgt"][:,indlev500,:,:]

In [29]:
print(getsizeof(geo)*1e-6) #in megabytes
print(geo.shape)

36.20348
(861, 73, 144)


In [31]:
# ahora podemos buscar los dos puntos que no interessan
Northpoint = (-22, 64)
Southpoint = (-25, 38)

#encontrar la position de la latitude y longitude correspondiente a estos punto
latNorth = (np.abs(lat-Northpoint[1])).argmin()
lonNorth = (np.abs(lon-Northpoint[0])).argmin()

latSouth = (np.abs(lat-Southpoint[1])).argmin()
lonSouth = (np.abs(lon-Southpoint[0])).argmin()

geoNorth = fileTmax.variables["hgt"][:,indlev500,latNorth,lonNorth]
geoSouth = fileTmax.variables["hgt"][:,indlev500,latNorth,lonNorth]

print(getsizeof(geoNorth)*1e-6) #in megabytes
print(getsizeof(geoSouth)*1e-6) #in megabytes


0.003564
0.003564


A veces tambien interessa selecionar una region entera, ejemplo, selecionamos el este de europa


In [32]:
box = (10, 40, 30, 80) #(lon1, lon2, lat1, lat2)
latbox = np.where((lat > box[2])&(lat<box[3]))[0]
lonbox = np.where((lon > box[0])&(lon<box[1]))[0]        

geobox = fileTmax.variables["hgt"][:,indlev500,latbox,lonbox]
geobox.shape

(861, 19, 11)

En cambio cuando queremos selecionnar datos que sobrepassan el meridian de greenwitch los tenemos que carregar en dos partes

In [33]:
lon_bnds = [-20, 40]
lat_bnds = [30, 80]
#convitimos las listas en numpy array 
lon_bnds = np.array(lon_bnds)
lat_bnds = np.array(lat_bnds)
# if longitude is written in negative convert it in value from 0 to 360 
#(if it is consistent with the longitude of the file we want to read)
lon_bnds = (lon_bnds + 360)%360    

if lon_bnds[0]>lon_bnds[1]:
    #calculamos los limites de la caja al oeste de greenwitch
    lonbox1 = np.where((lon >= lon_bnds[0]))[0]
    #calculamos los limites de la caja al este de greenwitch
    lonbox2 = np.where((lon <= lon_bnds[1]))[0]
    #extraemos los datos del oeste de greenwitch
    geobox1 = filegeo.variables["hgt"][:,indlev500,latbox,lonbox1]
    #extraemos los datos del este de greenwitch
    geobox2 = filegeo.variables["hgt"][:,indlev500,latbox,lonbox2]
    #juntamos los dos cajas
    geobox = np.concatenate((geobox1, geobox2), axis=2)
    
else:
    lonbox = np.where((los >= lon_bnds[0]) & (lon <= lon_bnds[1]))[0]
    geobox = fileTmax.variables["hgt"][:,indlev500,latbox,lonbox]
geobox.shape    

(861, 19, 25)

Cuidado a veces hay problema con seleccionar partes discontinuas de un netcdf

Tambien a veces queremos solo una parte de la fechas

In [ ]:
after2000 = np.where(dates > datetime(2000, 1, 1))[0]
after2000

In [ ]:
geobox2000 = filegeo.variables["hgt"][after2000,indlev500,:,:]
geobox2000.shape

cuando acabamos con la lectura de los datos es importante cierrar el fichiero

In [ ]:
filegeo.close()

### Donde guardar el fichiero: PATH

Teneis varias opciones, lo mas facil es guardar el fichiero el la misma carpeta que la del notebook. 

Pero lo mas "limpio" es crear una carpeta dedicada donde guarderaris todos los datos, por ejemplo nombrado `data`. Despues podeis organizar vuestros datos por producto, es decir crear una carpeta `NCEP` y por variable `geo`.

Una vez habeis guardados el fichieros en el lugar que os paresca correcto, podeis hacer click derecho, propiedades para encontrar el PATH de fichiero.

despues se lee el fichiero de esta manera.

In [ ]:
pathtofile = "aqui poner el path de vuestros datos"
print(os.path.join(pathtofile, "hgt.mon.mean.nc"))

In [ ]:
fileTmax = nc(os.path.join(pathtofile, "hgt.mon.mean.nc"), "r") 

In [ ]:
fileTmax.close()